# Import requirements

In [1]:
import os
import random
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(f'Device: {DEVICE}')

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Device: cuda


# Data Augmentation Using `transforms`

In [2]:
train_dataset = datasets.EMNIST(root='../data/EMNIST',
                                 train=True,
                                 download=True,
                                 split = 'letters',
                                 transform=transforms.ToTensor(),
                                 
                                )
test_dataset = datasets.EMNIST(root='../data/EMNIST',
                                train=False,
                                download=True,
                                split = 'letters',
                                transform=transforms.ToTensor(),
                                )


Extracting ../data/EMNIST/EMNIST/raw/gzip.zip to ../data/EMNIST/EMNIST/raw


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
for image, label in train_dataset:
    print(image.shape)
    print(label)
    break

torch.Size([1, 28, 28])
23


In [4]:
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

# Define the Convolution Neural Network (CNN)

In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # input shape = (1, 28, 28)
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1, bias=False), # (28, 28)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1, bias=False), # (28, 28)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # (14, 14)
            
            nn.Conv2d(64, 128, 3, padding=1, bias=False), # (14, 14)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1, bias=False), # (14, 14)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # (7, 7)
        )


        self.clssify = nn.Sequential(
            nn.Linear(7*7*128, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 26),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 7*7*128)
        x = self.clssify(x)
        return x

# Define the train, evaluation

In [6]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    train_loss = 0
    correct = 0

    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        label -= 1

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % log_interval == 0:
            pct = 100 * batch_idx / len(train_loader) # percent
            train_loss /= log_interval
            print(f'Train Epoch: {Epoch} [{batch_idx * len(image)}/{len(train_loader.dataset)} ({pct:.0f}%)]\tAverage Train Loss: {train_loss:.6f}')
            train_loss = 0


def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            label -= 1
            
            output = model(image)
            test_loss += criterion(output, label).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)

    return test_loss, test_accuracy

# set seeds

In [7]:
def fix_seeds(seed = 42, use_torch=False):
    # fix the seed for reproducibility 
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

    if use_torch: 
        torch.manual_seed(seed) 
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

# initialize the weights

In [8]:
def init_weights(m):
    # initialize the weight, bias
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform_(m.weight.data)
        if m.bias is not None:
            torch.nn.init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight.data, mean=1, std=0.02)
        torch.nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight.data)
        torch.nn.init.normal_(m.bias.data)

# Train & Test the model

In [13]:
len(train_loader.dataset)

124800

In [15]:
SEED = 42
EPOCHS = 20
#DEVICE = torch.device('cpu')


fix_seeds(seed=SEED, use_torch=True)
model = ConvNet().to(device=DEVICE)
model.apply(init_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)
criterion = nn.CrossEntropyLoss()


for Epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval=len(train_loader) * 0.2)
    test_loss, test_acc = evaluate(model, test_loader)
    scheduler.step(test_loss)

    print(f'\nEpoch: {Epoch}')
    print(f'Average Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_acc:.2f} %\n')
    #torch.save(model, f'./models/model_{Epoch:02d}.pt')

Train Epoch: 1 [24928/124800 (20%)]	Average Train Loss: 1.281469
Train Epoch: 1 [49888/124800 (40%)]	Average Train Loss: 0.567666
Train Epoch: 1 [74848/124800 (60%)]	Average Train Loss: 0.454784
Train Epoch: 1 [99808/124800 (80%)]	Average Train Loss: 0.393729
Train Epoch: 1 [124768/124800 (100%)]	Average Train Loss: 0.360165

Epoch: 1
Average Test Loss: 0.0068
Test Accuracy: 92.76 %

Train Epoch: 2 [24928/124800 (20%)]	Average Train Loss: 0.318757
Train Epoch: 2 [49888/124800 (40%)]	Average Train Loss: 0.298382
Train Epoch: 2 [74848/124800 (60%)]	Average Train Loss: 0.278921
Train Epoch: 2 [99808/124800 (80%)]	Average Train Loss: 0.263882
Train Epoch: 2 [124768/124800 (100%)]	Average Train Loss: 0.251011

Epoch: 2
Average Test Loss: 0.0058
Test Accuracy: 93.82 %

Train Epoch: 3 [24928/124800 (20%)]	Average Train Loss: 0.219814
Train Epoch: 3 [49888/124800 (40%)]	Average Train Loss: 0.218117
Train Epoch: 3 [74848/124800 (60%)]	Average Train Loss: 0.220862
Train Epoch: 3 [99808/124800 (8